# Os Silkers

In [ ]:
from load_history import load_history
from train import train
from predict import predict

In [ ]:
df, original_df = load_history('/home/guilherme/code/python/OsSilkers/data/parquet')
df

In [ ]:
model = train(df)

In [ ]:
predictions = predict(1, 1, 2023, model, '.')

In [ ]:
predictions.to_parquet('../saved/first_pred.parquet')